# ANN and CNN modeling - NestedCV preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.8 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/

/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA


In [ ]:
import numpy as np
import os
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, BatchNormalization,
                                   Dropout, Dense, Flatten,
                                   Input, Concatenate)
import keras_tuner as kt
from tensorflow.keras import backend as K
import gc

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import time
import pandas as pd

In [ ]:
seed = 0
tf.random.set_seed(seed)

In [ ]:
#X_images = np.load(r'C:\Users\andre\Desktop\Stage 2A\Projet\preprocessed_data\nv_mel_ratio\X_segmented.npy', allow_pickle=True)
X_align = np.load(r'/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/nv_mel_pca/X_align.npy', allow_pickle=True)
masks = np.load(r'/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/nv_mel_pca/masks.npy', allow_pickle=True)
y = np.load(r'/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/nv_mel_pca/y.npy')

In [ ]:
masks.shape

(7793, 450, 600)

## Create NN with shape information

In [ ]:
y_encoded = np.abs(np.ones(y.shape) - LabelEncoder().fit_transform(y)).astype(int)

In [ ]:
if len(y_encoded[y_encoded==1]) > len(y_encoded[y_encoded==0]):
  print("Label Mapping: 'nv': 1, 'mel': 0")
else:
  print("Label Mapping: 'nv': 0, 'mel': 1")

Label Mapping: 'nv': 0, 'mel': 1


In [ ]:
def calculate_class_weights(y):
    #Calculate weights
    weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y),
        y=y
    )
    #Create dictionary mapping class indices to weights
    class_weights = dict(zip(range(len(weights)), weights))
    return class_weights


In [ ]:
print(calculate_class_weights(y_encoded))

{0: np.float64(0.5829593058049073), 1: np.float64(3.5135256988277725)}


# Define model spaces

## CNN models

In [ ]:
def build_cnn_model(hp):
    num_layers = hp.Int('num_layers', 2, 4)
    l2_reg = hp.Float('reg', 1e-4, 1e-2, sampling='log')
    lr = hp.Float('learning_rate', 0.0005, 0.002, sampling='log')
    dropout_rate = hp.Float('dropout_rate', 0.1, 0.4, step=0.1)
    inputs = tf.keras.layers.Input(shape=(masks.shape[1], masks.shape[2], 1))
    x = inputs

    for i in range(num_layers):
        units = hp.Choice(f'filter_units_{i}', [16, 32, 64])
        x = tf.keras.layers.Conv2D(filters=units, kernel_size=(3, 3), activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.MaxPooling2D((2, 2))(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    x = tf.keras.layers.Flatten()(x)

    #Dense layers
    num_layers_dense = hp.Int('num_layers_dense', 1, 3)
    dropout_dense = hp.Float('dropout_dense', 0.1, 0.4, step=0.1)

    for i in range(num_layers_dense):
        units = hp.Choice(f'dense_units_{i}', [32, 64, 128])
        x = tf.keras.layers.Dense(units, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_dense)(x)

    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model


## ANN models

In [ ]:
def build_ann_model(hp):
    num_layers = hp.Int('num_layers', 1, 3)
    l2_reg = hp.Float('l2_reg', 1e-4, 1e-2, sampling='log')
    dropout_rate = hp.Float('dropout_rate', 0.1, 0.4, step=0.1)
    lr = hp.Float('learning_rate', 0.0005, 0.002, sampling='log')
    inputs = tf.keras.layers.Input(shape=(X_align.shape[1],))
    x = inputs

    for i in range(num_layers):
        units = hp.Choice(f'dense_units_{i}', [16, 32, 64, 128])
        x = tf.keras.layers.Dense(units, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l2_reg))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )
    return model


# Model robust evaluation - Nested Stratified KFold CV with BayesianOptimization

In [ ]:
CNN_metrics = {'auc': [], 'balanced_accuracy': [], 'f1': [], 'training_time':[], 'FPR':[], 'TPR':[], 'Threshold':[]}
ANN_metrics = {'auc': [], 'balanced_accuracy': [], 'f1': [], 'training_time':[], 'FPR':[], 'TPR':[], 'Threshold':[]}
best_hps_CNN_per_fold = []
best_hps_ANN_per_fold = []
execution_time_CNN = 0
execution_time_ANN = 0
n_inner_splits = 2
n_outer_splits = 20

## define CV

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, f1_score

#define stratified splits
skf_external = StratifiedKFold(n_splits=n_outer_splits, shuffle=True, random_state=seed)

for fold_idx, (train_idx, test_idx) in enumerate(skf_external.split(masks, y)):
    if fold_idx >= 4: #split the work between 3 notebooks
      continue
    print(f"Fold externe {fold_idx+1}/20")
    X_train_CNN, X_test_CNN = masks[train_idx], masks[test_idx]
    X_train_ANN, X_test_ANN = X_align[train_idx], X_align[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]

    #we clear memory at the start of each external fold
    tf.keras.backend.clear_session()
    gc.collect()

    #Inner tuner (RandomSearch optimization)
    start_CNN = time.time()

    tuner_CNN = kt.RandomSearch(
        build_cnn_model,
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=10,
        seed = seed + fold_idx,
        overwrite=True,
        directory=f'tuner_cnn_fold_{fold_idx}',  # Dossier unique par fold
        project_name='cnn_search'
    )
    end_CNN = time.time()
    execution_time_CNN += end_CNN - start_CNN

    start_ANN = time.time()
    tuner_ANN = kt.RandomSearch(
        build_ann_model,
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=10,
        seed = seed + fold_idx,
        overwrite=True,
        directory=f'tuner_ann_fold_{fold_idx}',  # Dossier unique par fold
        project_name='ann_search'
    )
    end_ANN = time.time()
    execution_time_ANN += end_ANN - start_ANN

    #define inner CV
    skf_internal = StratifiedKFold(n_splits=n_inner_splits, shuffle=True, random_state=seed)
    #sauvegarde des métriques
    ann_auc_inner = [[] for _ in range(n_inner_splits)]
    cnn_auc_inner = [[] for _ in range(n_inner_splits)]

    for inner_fold_idx, (inner_train_idx, inner_val_idx) in enumerate(skf_internal.split(X_train_CNN, y_train)):
        print(f"Fold interne {inner_fold_idx+1}/2")
        X_inner_train_CNN, X_inner_val_CNN = X_train_CNN[inner_train_idx], X_train_CNN[inner_val_idx]
        X_inner_train_ANN, X_inner_val_ANN = X_train_ANN[inner_train_idx], X_train_ANN[inner_val_idx]
        y_inner_train, y_inner_val = y_train[inner_train_idx], y_train[inner_val_idx]

        # standardize data
        start_ANN = time.time()
        scaler = StandardScaler()
        X_inner_train_ANN = scaler.fit_transform(X_inner_train_ANN)
        X_inner_val_ANN = scaler.transform(X_inner_val_ANN)


        # class weights
        start_CNN = time.time()
        class_weights_inner_train = calculate_class_weights(y_inner_train)
        end = time.time()
        execution_time_ANN += end - start_ANN
        execution_time_CNN += end - start_CNN

        # Recherche sur inner folds lors de la PREMIERE itération
        if inner_fold_idx == 0:
            print("Recherche des hyperparamètres...")

            # ANN
            start_ANN = time.time()
            tuner_ANN.search(
                X_inner_train_ANN, y_inner_train,
                validation_data=(X_inner_val_ANN, y_inner_val),
                epochs=50,
                batch_size=32,
                class_weight=class_weights_inner_train,
                callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                verbose=2
            )

            end = time.time()
            execution_time_ANN += end - start_ANN

            # Clear après ANN
            tf.keras.backend.clear_session()
            gc.collect()


            # CNN
            start_CNN = time.time()
            tuner_CNN.search(
                X_inner_train_CNN, y_inner_train,
                validation_data=(X_inner_val_CNN, y_inner_val),
                epochs=50,
                batch_size=32,
                class_weight=class_weights_inner_train,
                callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                verbose=2
            )
            end = time.time()
            execution_time_CNN += end - start_CNN

            # on récupère tous les configurations explorées
            all_trials_CNN = tuner_CNN.oracle.trials.copy()  # Copie pour éviter références
            all_trials_ANN = tuner_ANN.oracle.trials.copy()

            # Mem clear
            del tuner_CNN
            del tuner_ANN
            tf.keras.backend.clear_session()
            gc.collect()


            # on sauvegarde l'auc pour chacune des configurations
            for _, trial in all_trials_CNN.items():
                cnn_auc_inner[inner_fold_idx].append(trial.score)
            for _, trial in all_trials_ANN.items():
                ann_auc_inner[inner_fold_idx].append(trial.score)

        # lors des itérations suivantes
        else:
            print(f"Évaluation des configs sur fold interne {inner_fold_idx+1}")

            # CNN d'abord
            for i, trial in enumerate(all_trials_CNN.values()):
                print(f"  CNN config {i+1}/{len(all_trials_CNN)}")
                hp = kt.engine.hyperparameters.HyperParameters()
                for k, v in trial.hyperparameters.values.items():
                    hp.Fixed(k, v)

                # Construire et entraîner
                start_CNN = time.time()
                model_cnn = build_cnn_model(hp)
                model_cnn.fit(
                    X_inner_train_CNN, y_inner_train,
                    validation_data=(X_inner_val_CNN, y_inner_val),
                    epochs=50,
                    batch_size=32,
                    class_weight=class_weights_inner_train,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                    verbose=0  # Moins verbose pour clarté
                )

                # Évaluation
                score = model_cnn.evaluate(X_inner_val_CNN, y_inner_val, verbose=0)[1]
                cnn_auc_inner[inner_fold_idx].append(score)
                end = time.time()
                execution_time_CNN += end - start_CNN

                # CRUCIAL: Libérer la mémoire immédiatement
                del model_cnn
                tf.keras.backend.clear_session()
                gc.collect()

            # ANN ensuite
            for i, trial in enumerate(all_trials_ANN.values()):
                print(f"  ANN config {i+1}/{len(all_trials_ANN)}")
                hp = kt.engine.hyperparameters.HyperParameters()
                for k, v in trial.hyperparameters.values.items():
                    hp.Fixed(k, v)

                # Construire et entraîner
                start_ANN = time.time()
                model_ann = build_ann_model(hp)
                model_ann.fit(
                    X_inner_train_ANN, y_inner_train,
                    validation_data=(X_inner_val_ANN, y_inner_val),
                    epochs=50,
                    batch_size=32,
                    class_weight=class_weights_inner_train,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
                    verbose=0
                )

                # Évaluation
                score = model_ann.evaluate(X_inner_val_ANN, y_inner_val, verbose=0)[1]
                ann_auc_inner[inner_fold_idx].append(score)
                end = time.time()
                execution_time_ANN += end - start_ANN

                # CRUCIAL: Libérer la mémoire immédiatement
                del model_ann
                tf.keras.backend.clear_session()
                gc.collect()


    #Calcul de la moyenne des auc
    ann_auc_inner = np.array(ann_auc_inner)
    cnn_auc_inner = np.array(cnn_auc_inner)
    ann_auc_mean = np.mean(ann_auc_inner, axis=0)
    cnn_auc_mean = np.mean(cnn_auc_inner, axis=0)

    ### - On récupère la meilleure config d'hyperparamètres trouvée - ###
    #CNN
    start_CNN = time.time()
    best_auc_cnn = -1
    best_hp_cnn = None
    for trial_id, auc_score in zip(all_trials_CNN.keys(), cnn_auc_mean):
        if auc_score > best_auc_cnn:
            best_auc_cnn = auc_score
            best_hp_cnn = all_trials_CNN[trial_id].hyperparameters
    end = time.time()
    execution_time_CNN += end - start_CNN

    #ANN
    start_ANN = time.time()
    best_auc_ann = -1
    best_hp_ann = None
    for trial_id, auc_score in zip(all_trials_ANN.keys(), ann_auc_mean):
        if auc_score > best_auc_ann:
            best_auc_ann = auc_score
            best_hp_ann = all_trials_ANN[trial_id].hyperparameters
    end = time.time()
    execution_time_ANN += end - start_ANN

    # Sauvegarde de ces hyperparamètres
    best_hps_CNN_per_fold.append(best_hp_cnn.values)
    best_hps_ANN_per_fold.append(best_hp_ann.values)

    #IMPORTANT: Nettoyer les tuners avant l'entraînement final
    del all_trials_CNN
    del all_trials_ANN
    tf.keras.backend.clear_session()
    gc.collect()

    print("Entraînement final sur tout le train externe...")

    #Training sur tout le train externe avec les meilleurs hps
    start = time.time()
    class_weights_train = calculate_class_weights(y_train)
    end = time.time()
    execution_time_ANN += end - start
    execution_time_CNN += end - start

    # Évaluation sur le test externe
    class_weights_external_test = calculate_class_weights(y_test)
    sample_weights_external_test = np.array([class_weights_external_test[y_val] for y_val in y_test])

    #CNN
    start = time.time()
    model_cnn_final = build_cnn_model(best_hp_cnn)
    model_cnn_final.fit(
        X_train_CNN, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_CNN, y_test),
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                patience=5,
                mode='min',
                restore_best_weights=True,
                verbose=0
            )
        ],
        class_weight=class_weights_train,
        verbose=2
    )
    y_pred_proba = model_cnn_final.predict(X_test_CNN)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0).flatten()

    #AUC_CNN
    auc_CNN = model_cnn_final.evaluate(X_test_CNN, y_test, verbose=0)[1]
    #Balanced_accuracy_CNN
    balanced_accuracy_CNN = balanced_accuracy_score(y_test, y_pred, sample_weight=sample_weights_external_test)
    #f1_score
    f1_CNN = f1_score(y_test, y_pred, average='weighted')
    #ROC_CNN
    roc_curve_CNN = roc_curve(y_test, y_pred_proba)
    fpr_CNN, tpr_CNN, thresholds_CNN = roc_curve_CNN

    end = time.time()
    execution_time_CNN += end - start
    CNN_metrics['training_time'].append(end - start)

    #Standardise les features pour l'ANN
    start_ANN = time.time()
    scaler = StandardScaler()
    X_train_ANN_scaled = scaler.fit_transform(X_train_ANN)
    X_test_ANN_scaled = scaler.transform(X_test_ANN)

    #ANN
    model_ann_final = build_ann_model(best_hp_ann)
    model_ann_final.fit(
        X_train_ANN_scaled, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_ANN_scaled, y_test),
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                patience=5,
                mode='min',
                restore_best_weights=True,
                verbose=0
            )
        ],
        class_weight=class_weights_train,
        verbose=2
      )
    y_pred_proba = model_ann_final.predict(X_test_ANN_scaled)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0).flatten()
    #AUC_ANN
    auc_ANN = model_ann_final.evaluate(X_test_ANN_scaled, y_test, verbose=0)[1]
    #Balanced_accuracy_ANN
    balanced_accuracy_ANN = balanced_accuracy_score(y_test, y_pred, sample_weight=sample_weights_external_test)
    #f1_ANN
    f1_ANN = f1_score(y_test, y_pred, average='weighted')
    #ROC_ANN
    roc_curve_ANN = roc_curve(y_test, y_pred_proba)
    fpr_ANN, tpr_ANN, thresholds_ANN = roc_curve_ANN

    end = time.time()
    execution_time_ANN += end - start_ANN
    ANN_metrics['training_time'].append(end - start_ANN)



    #### - CNN metrics - ####

    CNN_metrics['auc'].append(auc_CNN)
    CNN_metrics['balanced_accuracy'].append(balanced_accuracy_CNN)
    CNN_metrics['f1'].append(f1_CNN)
    CNN_metrics['FPR'].append(fpr_CNN)
    CNN_metrics['TPR'].append(tpr_CNN)
    CNN_metrics['Threshold'].append(thresholds_CNN)

    #### - ANN metrics - ####

    ANN_metrics['auc'].append(auc_ANN)
    ANN_metrics['balanced_accuracy'].append(balanced_accuracy_ANN)
    ANN_metrics['f1'].append(f1_ANN)
    ANN_metrics['FPR'].append(fpr_ANN)
    ANN_metrics['TPR'].append(tpr_ANN)
    ANN_metrics['Threshold'].append(thresholds_ANN)

    #Nettoyer après chaque fold externe
    del model_cnn_final
    del model_ann_final
    tf.keras.backend.clear_session()
    gc.collect()



# Sauvegarde finale des résultats
df_CNN_metrics = pd.DataFrame(CNN_metrics)
df_ANN_metrics = pd.DataFrame(ANN_metrics)

df_CNN_metrics.to_csv("/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/CNN_metrics_1-4.csv", index=False)
df_ANN_metrics.to_csv("/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/ANN_metrics_1-4.csv", index=False)

pd.DataFrame(best_hps_CNN_per_fold).to_csv("/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/best_hps_CNN_1-4.csv", index=False)
pd.DataFrame(best_hps_ANN_per_fold).to_csv("/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/best_hps_ANN_1-4.csv", index=False)

print("DONE")

Trial 10 Complete [00h 02m 13s]
val_auc: 0.7151142358779907

Best val_auc So Far: 0.7295193076133728
Total elapsed time: 00h 17m 29s
Fold interne 2/2
Évaluation des configs sur fold interne 2
  CNN config 1/10
  CNN config 2/10
  CNN config 3/10
  CNN config 4/10
  CNN config 5/10
  CNN config 6/10
  CNN config 7/10
  CNN config 8/10
  CNN config 9/10
  CNN config 10/10
  ANN config 1/10
  ANN config 2/10
  ANN config 3/10
  ANN config 4/10
  ANN config 5/10
  ANN config 6/10
  ANN config 7/10
  ANN config 8/10
  ANN config 9/10
  ANN config 10/10
Entraînement final sur tout le train externe...
Epoch 1/50
232/232 - 29s - 124ms/step - auc: 0.6275 - loss: 2.7453 - val_auc: 0.6562 - val_loss: 1.6917
Epoch 2/50
232/232 - 6s - 27ms/step - auc: 0.6629 - loss: 1.7020 - val_auc: 0.5231 - val_loss: 1.2881
Epoch 3/50
232/232 - 6s - 27ms/step - auc: 0.6926 - loss: 1.3421 - val_auc: 0.5248 - val_loss: 1.0060
Epoch 4/50
232/232 - 6s - 27ms/step - auc: 0.7154 - loss: 1.1805 - val_auc: 0.6909 - val_l

In [ ]:
print(f"Temps d'execution CNN : {execution_time_CNN} secondes")
print(f"Temps d'execution ANN : {execution_time_ANN} secondes")

#save those times
with open('/content/drive/MyDrive/Stage 2A/ANN_CNN_comparison_PCA/execution_time_1.txt', 'w') as f:
    f.write("CNN " + str(execution_time_CNN) + 's')
    f.write('\n')
    f.write("ANN " + str(execution_time_ANN) + 's')


Temps d'execution CNN : 9741.59652686119 secondes
Temps d'execution ANN : 2049.7476539611816 secondes
